# 测试集数据预处理
这一部分的过程与训练集的数据处理基本一致，这里不再赘述。有区别的地方在于可能官方给的测试集的字段与训练集不完全一致，所以使用本部分代码时需要注意

# 读数据

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

# 获取当前的工作目录
pwd = os.getcwd()
# 将工作目录更改到测试集
os.chdir("原始测试集")
# ——————————————————————————读取数据—————————————————————————————— #
# 航班数据
flight_data = pd.read_csv('flight.csv',sep=',',encoding='gb2312')
# 天气数据
weather = pd.read_excel('weather.xlsx')
# 城市与机场对应数据
airport_city = pd.read_excel('airport_city.xlsx')
# 特情
spcial = pd.read_excel('spcial.xlsx')
# 天气情况
case = pd.read_csv('weather_case.csv',encoding='gb2312')
# 改回原来的工作目录
os.chdir(pwd)
flight_data['标识'] = flight_data['验证标识（1：需要选手预测；0：提前两小时航班信息参考数据）']
del(flight_data['验证标识（1：需要选手预测；0：提前两小时航班信息参考数据）'])

## 纠正部分字段错误，没错误不需要纠正

In [5]:
flight_data['航班编号1'] = flight_data['飞机编号']
flight_data['飞机编号'] = flight_data['航班编号']
flight_data['航班编号'] = flight_data['航班编号1']
del(flight_data['航班编号1'])

In [3]:
flight_data.head()

,出发机场,到达机场,飞机编号,计划出发时间,计划到达时间,实际出发时间,实际到达时间,航班编号,取消标识,标识
0,XIY,CKG,2403.0,1501545000,1501551000,NaN,NaN,MU2261,NaN,1
1,PVG,KMG,1126.0,1501544400,1501556400,NaN,NaN,CZ3677,NaN,1
2,SJW,NKG,448.0,1501545300,1501551300,NaN,NaN,9C8939,NaN,1
3,XIY,NKG,779.0,1501545300,1501552200,NaN,NaN,MU2387,NaN,1
4,TAO,PVG,948.0,1501543800,1501548900,NaN,NaN,MU5573,NaN,1


# 时间信息预处理

In [6]:
# # 转化成日期格式
flight_data['计划起飞时间1'] = pd.to_datetime(flight_data['计划出发时间'],unit='s',utc=True)
flight_data['计划到达时间1'] = pd.to_datetime(flight_data['计划到达时间'],unit='s',utc=True)
# # 计划飞行时间
flight_data['计划飞行时间'] = flight_data['计划到达时间1'] - flight_data['计划起飞时间1']
flight_data['计划飞行时间'] = flight_data['计划飞行时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)

flight_data['计划飞行时间'] = flight_data['计划飞行时间']/3600  # 转换为小时
# 细分时间段
flight_data['计划起飞日期'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划起飞时刻'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
flight_data['航班月份'] = flight_data['计划起飞时间1'].apply(lambda x:int(x.strftime('%m')) if not(pd.isnull(x)) else None)

flight_data['计划到达日期'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划到达时刻'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
##延误
flight_data['起飞延误时间'] = pd.to_datetime(flight_data['实际出发时间'],unit='s',utc=True) - pd.to_datetime(flight_data['计划出发时间'],unit='s',utc=True)
flight_data['起飞延误时间'] = flight_data['起飞延误时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
flight_data['起飞延误时间'] = flight_data['起飞延误时间']/3600  # 转换为分钟
flight_data['起飞延误时间'] = np.where(flight_data['取消标识'] == '取消',10,flight_data['起飞延误时间'])

flight_data['到达延误时间'] = pd.to_datetime(flight_data['实际到达时间'],unit='s',utc=True) - pd.to_datetime(flight_data['计划到达时间'],unit='s',utc=True)
flight_data['到达延误时间'] = flight_data['到达延误时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
flight_data['到达延误时间'] = flight_data['到达延误时间']/3600  # 转换为分钟
flight_data['到达延误时间'] = np.where(flight_data['取消标识'] == '取消',10,flight_data['到达延误时间'])

del flight_data['计划起飞时间1']
del flight_data['计划到达时间1']
del(flight_data['实际出发时间'])
del(flight_data['取消标识'])

In [6]:
flight_data.head()

,出发机场,到达机场,飞机编号,计划出发时间,计划到达时间,实际到达时间,航班编号,标识,计划飞行时间,计划起飞日期,计划起飞时刻,航班月份,计划到达日期,计划到达时刻,起飞延误时间,到达延误时间
0,XIY,CKG,2403.0,1501545000,1501551000,NaN,MU2261,1,1.666667,2017-07-31,23,7,2017-08-01,01,NaN,NaN
1,PVG,KMG,1126.0,1501544400,1501556400,NaN,CZ3677,1,3.333333,2017-07-31,23,7,2017-08-01,03,NaN,NaN
2,SJW,NKG,448.0,1501545300,1501551300,NaN,9C8939,1,1.666667,2017-07-31,23,7,2017-08-01,01,NaN,NaN
3,XIY,NKG,779.0,1501545300,1501552200,NaN,MU2387,1,1.916667,2017-07-31,23,7,2017-08-01,01,NaN,NaN
4,TAO,PVG,948.0,1501543800,1501548900,NaN,MU5573,1,1.416667,2017-07-31,23,7,2017-08-01,00,NaN,NaN


# 前序航班的延误时间&到达与起飞间隔
这里前序航班的定义为：同一架飞机，当前航班的前一个航班即为当前航班的前序航班。比如，同一架飞机连续飞两个航班A：南京--北京，B：北京--西安，则A为B的前序航班。
## 延误时间（单位：h）
前序航班的延误时间定义为前序航班到达延误时间，即时间到达时间减去计划到达时间
## 到达与起飞间隔（单位：h）
当前航班的计划起飞时间与前序航班时间到达时间的间隔

In [14]:
flight_data['飞机编号']= flight_data['飞机编号'].fillna(0)
flight_data['前序延误'] = pd.Series()
flight_data['起飞间隔'] = pd.Series()
grouped = flight_data.groupby(flight_data['飞机编号'])
chunks = []
for name,group in grouped:
    group = group.sort_values('计划出发时间')
    a = pd.to_datetime(group['计划出发时间'],unit='s',utc=True)[1:].reset_index(drop=True)
    b = pd.to_datetime(group['实际到达时间'],unit='s',utc=True)[0:len(group)-1].reset_index(drop=True)  
    group['起飞间隔'][1:] = a-b
    group['起飞间隔'] = group['起飞间隔'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
    group['起飞间隔'] = group['起飞间隔']/3600
    
    group['前序延误'][1:] = group['到达延误时间'][0:len(group)-1]
    chunks.append(group)
flight_data = pd.concat(chunks, ignore_index=True) 
del(grouped)
del(group)
del(chunks)
flight_data['前序延误'][flight_data['飞机编号']==0] = np.NaN
flight_data['起飞间隔'][flight_data['飞机编号']==0] = np.NaN
flight_data = flight_data[flight_data['标识']==1]

In [6]:
flight_data.head()

,出发机场,到达机场,飞机编号,计划出发时间,计划到达时间,实际到达时间,航班编号,标识,计划飞行时间,计划起飞日期,计划起飞时刻,航班月份,计划到达日期,计划到达时刻,起飞延误时间,到达延误时间,前序延误,起飞间隔
0,HRB,TNA,0.0,1501542000,1501550400,NaN,SC8727,1,2.333333,2017-07-31,23,7,2017-08-01,01,NaN,NaN,NaN,NaN
1,HRB,NNG,0.0,1501542000,1501564200,NaN,SC8727,1,6.166667,2017-07-31,23,7,2017-08-01,05,NaN,NaN,NaN,NaN
2,XMN,HSN,0.0,1501542300,1501547400,NaN,SC4967,1,1.416667,2017-07-31,23,7,2017-08-01,00,NaN,NaN,NaN,NaN
3,XMN,TAO,0.0,1501542300,1501555500,NaN,SC4967,1,3.666667,2017-07-31,23,7,2017-08-01,02,NaN,NaN,NaN,NaN
4,KMG,ZUH,0.0,1501542900,1501549500,NaN,8L9893,1,1.833333,2017-07-31,23,7,2017-08-01,01,NaN,NaN,NaN,NaN


# 特情
特情这个特征的处理，没有区分特情的具体内容，只将特情发生的时间段对应到计划起飞和到达的时间，以0代表没有发生特情，1表示发生了特情，所以后面有继续优化这个特征的空间

In [8]:
# --------------------------------------------------特情处理------------------------------------------#
del(spcial['采集时间'])
del(spcial['特情'])
spcial['开始日期'] = spcial['开始时间'].apply(lambda x : x.strftime('%F'))
spcial['开始时刻'] = spcial['开始时间'].apply(lambda x : x.strftime('%H'))
spcial['结束时刻'] = spcial['结束时间'].apply(lambda x : x.strftime('%H'))

del(spcial['开始时间'])
del(spcial['结束时间'])
spcial = spcial.drop_duplicates(['机场','开始日期'])
flight_data = pd.merge(flight_data,spcial,left_on=['到达机场','计划到达日期'],right_on=['机场','开始日期'],how='left',sort=False)

flight_data['到达特情'] = np.where((flight_data['计划到达时刻'] >=flight_data['开始时刻']) &
                               (flight_data['计划到达时刻']<= flight_data['结束时刻']),1,0)
del(flight_data['机场'])
del(flight_data['开始日期'] )
del(flight_data['开始时刻'])
del(flight_data['结束时刻'] )
flight_data = pd.merge(flight_data,spcial,left_on=['出发机场','计划起飞日期'],right_on=['机场','开始日期'],how='left',sort=False)

flight_data['出发特情'] = np.where((flight_data['计划起飞时刻'] >=flight_data['开始时刻']) &
                               (flight_data['计划起飞时刻']<= flight_data['结束时刻']),1,0)
del(flight_data['机场'])
del(flight_data['开始日期'] )
del(flight_data['开始时刻'])
del(flight_data['结束时刻'] )

# 天气
天气特征的提取主要包括气温特征和天气情况，其中：

1. 气温划分为3个取值，大于40度为高温，小于-10度为低温，其他为一般
2. 天气情况（小雨、阴天等）根据组织方提供的天气信息文件，两年时间内，所有时间所有地点出现频率小于50的天气统一划归为‘other’

In [9]:
# # ———————————————————————————天气数据预处理———————————————————— #
weather['日期'] = weather['日期'].apply(lambda x : x.strftime('%F'))
weather['气温'] = pd.Series()
weather['最高气温'] = weather['最高气温'].fillna('0')
weather['最低气温'] = weather['最低气温'].fillna('0')
weather['气温'] = weather['最高气温'].apply(lambda x: '高温' if int(x)>=40 else '一般')
weather['气温'] = np.where(weather['最低气温'].astype('int') < -10,'低温',weather['气温'])
del(weather['最高气温'])
del(weather['最低气温'])
weather = weather.drop_duplicates() # 只包含这三个字段

weather_case = list(case['0'])
weather['天气'] = weather['天气'].apply(lambda x: x if x in set(weather_case) else 'other')
# # 将机场编码对应到天气数据上面，根据城市名
airport_weather = pd.merge(weather,airport_city,left_on=['城市'],right_on=['城市名称'],how='left',sort=False)
del(airport_weather['城市名称'])
# 去除缺失值和重复的机场天气信息
airport_weather = airport_weather.dropna()
airport_weather = airport_weather.drop_duplicates(['日期','机场编码'])

In [19]:
airport_weather.head()

,城市,天气,日期,气温,机场编码
17,忻州,阵雨,2017-08-01,一般,WUT
26,大连,多云转阵雨,2017-08-01,一般,DLC
30,汉中,晴,2017-08-01,一般,HZG
33,潍坊,阵雨转多云,2017-08-01,一般,WEF
35,博乐,晴,2017-08-01,一般,BPL


## 将天气匹配到航班动态表中

In [10]:
# 出发城市
flight_data = pd.merge(flight_data,airport_weather,left_on=['出发机场','计划起飞日期'],right_on=['机场编码','日期'],how='left',sort=False)
flight_data['出发天气'] = flight_data['天气']
flight_data['出发气温'] = flight_data['气温']
del(flight_data['天气'])
del(flight_data['机场编码'])
del(flight_data['城市'])
del[flight_data['日期']]
del(flight_data['气温'])
# 到达城市
flight_data = pd.merge(flight_data,airport_weather,left_on=['到达机场','计划到达日期'],right_on=['机场编码','日期'],how='left',sort=False)
flight_data['到达天气'] = flight_data['天气']
flight_data['到达气温'] = flight_data['气温']
del(flight_data['天气'])
del(flight_data['机场编码'])
del(flight_data['城市'])
del[flight_data['日期']]
del(flight_data['气温'])
del(flight_data['实际到达时间'])
del(flight_data['计划起飞日期'])
del(flight_data['到达延误时间'])
del(flight_data['计划到达日期'])
del(flight_data['标识'])

In [38]:
flight_data.head()

,出发机场,到达机场,飞机编号,计划出发时间,计划到达时间,实际到达时间,航班编号,标识,计划飞行时间,计划起飞日期,...,起飞间隔,到达特情,出发特情,城市,天气,最低气温,最高气温,日期,机场编码,出发天气
0,HRB,TNA,0.0,1501542000,1501550400,NaN,SC8727,1,2.333333,2017-07-31,...,NaN,0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,HRB,NNG,0.0,1501542000,1501564200,NaN,SC8727,1,6.166667,2017-07-31,...,NaN,0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,XMN,HSN,0.0,1501542300,1501547400,NaN,SC4967,1,1.416667,2017-07-31,...,NaN,0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,XMN,TAO,0.0,1501542300,1501555500,NaN,SC4967,1,3.666667,2017-07-31,...,NaN,0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,KMG,ZUH,0.0,1501542900,1501549500,NaN,8L9893,1,1.833333,2017-07-31,...,NaN,0,0,NaN,NaN,NaN,NaN,NaT,NaN,NaN


# 航空公司

In [11]:
# 航班性质为：0：补飞，1：国内正常，2：国外
flight_data['航空公司'] = flight_data['航班编号'].apply(lambda x : x[:2])
def f(x):
    if x[-1].isalpha():
        y = 0
    elif len(x[2:]) == 4:
        y = 1
    else:
        y = 2
    return(y)
flight_data['航班性质'] = flight_data['航班编号'].apply(f)

In [12]:
flight_data.head()

,出发机场,到达机场,飞机编号,计划出发时间,计划到达时间,航班编号,计划飞行时间,计划起飞时刻,航班月份,计划到达时刻,...,前序延误,起飞间隔,到达特情,出发特情,出发天气,出发气温,到达天气,到达气温,航空公司,航班性质
0,HRB,TNA,0.0,1501542000,1501550400,SC8727,2.333333,23,7,01,...,NaN,NaN,0,0,NaN,NaN,雷阵雨,一般,SC,1
1,HRB,NNG,0.0,1501542000,1501564200,SC8727,6.166667,23,7,05,...,NaN,NaN,0,0,NaN,NaN,阵雨,一般,SC,1
2,XMN,HSN,0.0,1501542300,1501547400,SC4967,1.416667,23,7,00,...,NaN,NaN,0,0,NaN,NaN,阵雨转阴,一般,SC,1
3,XMN,TAO,0.0,1501542300,1501555500,SC4967,3.666667,23,7,02,...,NaN,NaN,0,0,NaN,NaN,阵雨,一般,SC,1
4,KMG,ZUH,0.0,1501542900,1501549500,8L9893,1.833333,23,7,01,...,NaN,NaN,0,0,NaN,NaN,阵雨转多云,一般,8L,1


# 保存处理完的测试集

In [13]:
# 注意这个处理完的数据集保存在了当前文件夹下-----‘处理后测试集’
os.chdir("处理后测试集")
flight_data.to_csv('test_data1.csv',index=False) 
os.chdir(pwd)